# Primeira parte- Análise e tratamento dos dados fornecidos



## 1- Principais bibliotecas que foram utilizadas



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import joblib

- Para a Manipulação de dados utilizamos pandas;

- Na Visualização trabalhamos com as bibliotecas matplotlib e seaborn;

- No Machine learning, utilizamos ao sklearn;

- E para salvar os resultado da análise, joblib.

## 2- Carregamento dos Dados
- Leitura do arquivo CSV com tratamento de exceção:

In [ ]:
 try:
      df = pd.read_csv("campeonatos_futebol_atualizacao.csv")#Fizemos uma cópia dos dados para ficar no mesmo diretório e poder acessá-los com mais facilidade
      print("Dados carregados com sucesso!")
     
  except FileNotFoundError:
      print("Arquivo não encontrado. Verifique o caminho.")
      exit()

-Esse tratamento garante que o arquivo exista e lida com erros na leitura, e caso haja erro exibirá a mensagem "arquivo não encontrado.Verifique o caminho"

## 3- Tratamento de Dados

- Preenchimento de valores ausentes:


In [ ]:
for col in df.select_dtypes(include=['float64', 'int64']):
    df[col] = df[col].fillna(df[col].median())
for col in df.select_dtypes(include=['object']):
    df[col] = df[col].fillna(df[col].mode()[0])

- Preenchimento de valores ausentes:
- Substitui NaN por:

  - Mediana, se for numérico;

  - Moda, se for categórico.

Evita warnings futuros do pandas ao não usar inplace=True em cadeia.

### Remoção de outliers com IQR:

- O IQR(Intervalo Interquartil) é utilizado para achar valores chamados outliers, que são valores fora da curva que acabem por ser irreais e extrapolem a curva de valores esperados.Nesse caso utilizamos dois parâmetro o Q1 e Q3, onde:
- Q1 (25%): é o valor que separa os 25% menores dados.

- Q3 (75%): é o valor que separa os 25% maiores dados.

- IQR: intervalo do “meio” onde estão os 50% centrais dos dados.


In [ ]:
def get_outlier_indices(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[column] < lower) | (df[column] > upper)].index

outlier_indices = set()
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    outlier_indices.update(get_outlier_indices(df, col))

df = df.drop(index=outlier_indices).reset_index(drop=True)

- Assim, removendo valores extremos com base no Intervalo Interquartílico (IQR).


## 4- Análise Exploratória dos Dados

 Aqui analisamos visualmente as variáveis mais relevantes para entender padrões e relações entre os dados.
- Seleção das variáveis:

In [ ]:
variaveis = ['Chutes a gol 1', 'Gols 1', 'Posse 1(%)', 'Escanteios 1', 
             'Faltas 1', 'Cartões amarelos 1', 'Cartões vermelhos 1',
             'Chutes a gol 2', 'Gols 2', 'Posse 2(%)', 'Escanteios 2', 
             'Faltas 2', 'Cartões amarelos 2', 'Cartões vermelhos 2']

Essas são as features (características) usadas para entender o desempenho de cada time na partida.

- Mapa de calor de correlação entre as variáveis:

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df[variaveis].corr(), annot=True, cmap="coolwarm")
plt.title("Correlação entre Variáveis Selecionadas")
plt.show()

 - Usado para visualizar correlações numéricas entre variáveis.
 - Permite identificar colunas com forte relação entre si (ex: mais chutes = mais gols?).

- Distribuição de gols:

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.histplot(df['Gols 1'], bins=range(0,10), color='blue').set_title('Gols - Time da Casa')
plt.subplot(1,2,2)
sns.histplot(df['Gols 2'], bins=range(0,10), color='red').set_title('Gols - Time Visitante')
plt.tight_layout()
plt.show()

Gráficos de histograma para visualizar frequência de gols dos times da casa e visitante

- Criação da coluna de resultado:

In [ ]:
df['Resultado'] = df.apply(lambda x: 'Casa' if x['Gols 1'] > x['Gols 2'] else ('Empate' if x['Gols 1'] == x['Gols 2'] else 'Fora'), axis=1)


 - Cria uma nova coluna com o resultado final da partida baseado no número de gols:

   - 'Casa' se o time da casa venceu

   - 'Fora' se o visitante venceu

   - 'Empate' caso os gols sejam iguais

- Gráfico de distribuição dos resultados:

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='Resultado', hue='Resultado', order=['Casa', 'Empate', 'Fora'],
              palette={'Casa':'blue', 'Empate':'gray', 'Fora':'red'}, legend=False)
plt.title('Distribuição de Resultados')
plt.show()

Mostra quantas partidas terminaram com vitória da casa, empate ou vitória do visitante.

# Segunda parte- Utilização dos dados para o Machine Learning

## Machine learning
Aqui usamos técnicas de aprendizado de máquina para prever o resultado da partida com base nas estatísticas do jogo.
-  Definindo variáveis de entrada (X) e saída (y):

In [ ]:
X = df[variaveis]
y = df['Resultado']

    - X são as variáveis estatísticas do jogo.
 
    - y é o resultado que queremos prever.
- Divisão entre treino e teste:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- Separa os dados em:

  - 80% para treino

  - 20% para teste

Garante que o modelo aprenda com uma parte dos dados e seja avaliado com outra.

- Criação e treinamento do modelo:

In [ ]:
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

- Random Forest: modelo baseado em várias árvores de decisão.

- n_estimators=100: usa 100 árvores para fazer previsões. As árvores são um fluxograma, uma sequência de perguntas até chegar a uma resposta.

- Previsão:

In [ ]:
y_pred = modelo.predict(X_test)

O modelo prevê os resultados com base nos dados de teste.

- Avaliação do modelo:

In [ ]:
print("\\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("Acurácia:", accuracy_score(y_test, y_pred))


- classification_report: mostra métricas como precisão, recall e F1-score.

- accuracy_score: mostra o percentual de acertos.

- Matriz de confusão:

In [ ]:
cmp = confusion_matrix(y_test, y_pred, labels=['Casa', 'Empate', 'Fora'])
plt.figure(figsize=(6,4))
sns.heatmap(cmp, annot=True, fmt='d', cmap='Blues', xticklabels=['Casa', 'Empate', 'Fora'], yticklabels=['Casa', 'Empate', 'Fora'])
plt.title('Matriz de Confusão')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.show()


Mostra quantas vezes o modelo acertou ou errou cada tipo de resultado.

## Salvamento do Modelo Treinado
Basicamente como o nome desta cédula diz, é o comando que criará um novo arquivo contendo o resultado dessa predição futebolística.
- Salvando com joblib fica assim:
  

In [ ]:
joblib.dump(modelo, 'modelo_previsao_futebol.pkl')
print("\\nA analise foi salva como 'modelo_previsao_futebol.pkl'")

Esse foi um resumo sobre como foi estruturado nosso código, espero que esteja tudo esclarecido após você ter lido esse arquivo!